In [1]:
import time
import os


def read_proc_self_stat():
    """Read utime and stime from /proc/self/stat."""
    with open("/proc/self/stat", "r") as f:
        fields = f.read().split()
        utime = int(fields[13])  # 14th field: user mode jiffies
        stime = int(fields[14])  # 15th field: kernel mode jiffies
    return utime, stime


def measure_cpu_utilization(code_block, *args, **kwargs):
    """
    Measure average CPU utilization for a code block (function).
    code_block: function to execute
    args, kwargs: arguments to the function
    Returns: CPU utilization percentage
    """
    clk_tck = os.sysconf(os.sysconf_names["SC_CLK_TCK"])

    # Read initial CPU times and wall time
    utime_start, stime_start = read_proc_self_stat()
    wall_start = time.monotonic()

    # Run the code section
    code_block(*args, **kwargs)

    # Read final CPU times and wall time
    utime_end, stime_end = read_proc_self_stat()
    wall_end = time.monotonic()

    # Calculate CPU time used (in seconds)
    cpu_time = (utime_end - utime_start + stime_end - stime_start) / clk_tck

    # Calculate wall-clock time (in seconds)
    wall_time = wall_end - wall_start

    # Compute CPU utilization as a percentage
    utilization = (cpu_time / wall_time) * 100 if wall_time > 0 else 0.0
    return utilization


# Example usage:
def busy_work():
    # Example: CPU-intensive loop
    for _ in range(10**8):
        pass


util = measure_cpu_utilization(busy_work)
print(f"Average CPU utilization: {util:.2f}%")


Average CPU utilization: 100.00%


In [2]:
import sys
sys.path.append("/home/jovyan/jupyter_notebooks")

for p in sys.path:
    print(p)

from instrument_code import SystemMonitor

/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip
/usr/local/spark/python
/home/jovyan
/opt/conda/lib/python312.zip
/opt/conda/lib/python3.12
/opt/conda/lib/python3.12/lib-dynload

/opt/conda/lib/python3.12/site-packages
/home/jovyan/jupyter_notebooks


In [5]:
# Create monitor instance
monitor = SystemMonitor()

# Get current memory usage
print(f"Current RSS memory usage: {monitor.get_current_rss():.2f} MB")

# Take CPU snapshot
before_snapshot = monitor.snapshot_cpu()
print(f"CPU snapshot: {before_snapshot}")

# Simulate some work
for _ in range(10**8):
    pass

# Take another snapshot
after_snapshot = monitor.snapshot_cpu()
print(f"CPU snapshot after work: {after_snapshot}")

print(
    f"\nCPU utilization:{monitor.compute_cpu_usage(before_snapshot, after_snapshot):.2f}%"
)


Current RSS memory usage: 69.92 MB
CPU snapshot: (436, 4, 10336.61722025)
CPU snapshot after work: (596, 4, 10338.222676334)

CPU utilization:99.66%
